# Spatial zonation

**Pinned Environment:** [`envs/sc-charter.yaml`](../../envs/sc-charter.yaml)  

In [ ]:
import sys
from pathlib import Path
import os
import squidpy as sq
import cellcharter as cc
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import session_info

In [ ]:
plt.rcParams["figure.dpi"] = 150

In [ ]:
sys.path.append(str(Path.cwd().resolve().parents[1]))

from config.paths import BASE_DIR

h5ad_dir = BASE_DIR / "data/h5ad/export_10"
output_dir = BASE_DIR / "data/h5ad/export_10"
cellcharter_dir = BASE_DIR / "cellcharter"

input_file = h5ad_dir / "iec-subset-resolvi-v2.h5ad"
output_file = output_dir / "iec-subset-resolvi-cc-v2.h5ad"

output_dir.mkdir(parents=True, exist_ok=True)
cellcharter_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
adata = sc.read_h5ad(input_file)
adata

In [ ]:
adata.X[:5, :5].toarray()

### subset to IECs

In [ ]:
edata = adata[adata.obs["Class"] == "Epithelial"].copy()
edata

In [ ]:
def run_cellcharter(adata, output_dir, n_layers=3):

    print("starting neighbors and cell charter zone id")
    sq.gr.spatial_neighbors(
        adata,
        library_key="sample_id",
        coord_type="generic",
        delaunay=True,
        spatial_key="spatial",
    )

    cc.gr.remove_long_links(adata)

    cc.gr.aggregate_neighbors(
        adata,
        n_layers=n_layers,
        use_rep="X_scVI",
        out_key="X_cellcharter",
        sample_key="sample_id",
    )

    print("starting clustering with fixed k=3")
    model = cc.tl.Cluster(n_clusters=3)
    model.fit(adata, use_rep="X_cellcharter")
    adata.obs["epithelial_cc_3"] = model.predict(adata, use_rep="X_cellcharter")

    adata.write_h5ad(output_file, compression="gzip")

    return adata

In [ ]:
edata = run_cellcharter(edata, h5ad_dir)